## Importing Python modules

In [2]:
from twarc import Twarc2, expansions
import os
import pandas as pd
import requests
import time

## Input your Twitter Bearer Token

In [3]:
TOKEN = "YOUR TWITTER ACADEMIC TRACK BEARER TOKEN"

## Query Parameter (e.g., search keyword, time)

In [6]:
os.environ["BEARER_TOKEN"] = TOKEN
bearer_token = os.environ["BEARER_TOKEN"]

def auth():
    return os.environ.get("BEARER_TOKEN")

next_token = ""
def create_url():
    if next_token == "":
        return "https://api.twitter.com/2/tweets/search/all"
    else: 
        return "https://api.twitter.com/2/tweets/search/all?next_token={}".format(next_token)
    
search_url = create_url()
    
query_params = {'query': "(PRSA OR @PRSA OR #PRSA) lang:en", # search query
                'tweet.fields': "author_id,created_at,public_metrics,in_reply_to_user_id",
                'start_time': "2022-01-01T00:00:00Z", # data collection start time
                'end_time': '2022-01-03T00:00:00Z',   # data collection end time
                'max_results': 300,
                "expansions":"referenced_tweets.id"}

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def connect_to_endpoint(url, headers, params):
    response = requests.request("GET", search_url, headers=headers, params=params)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

## Crawling Start

In [7]:
df = pd.DataFrame() 
retweet_df_disk = pd.DataFrame() 

next_token = ''
full_text = ''
row = ''

count = 0

full_text_column = []
includes_id = []
includes_retweet = []

while True:
    bearer_token = auth()
    search_url = create_url()
    headers = create_headers(bearer_token)    
    json_response = connect_to_endpoint(search_url, headers, query_params)
    
    tweet = pd.DataFrame.from_dict(json_response['data']) 
    try:
        tweet_1 = tweet.explode('referenced_tweets')
        tweet_1 = tweet_1.referenced_tweets.apply(pd.Series)
        tweet_2 = tweet_1[["id", "type"]]
        tweet_2.columns = ['retweet_id', 'type']
        tweets = pd.concat([tweet, tweet_2], axis=1)
    
        ### data file
        df = df.append(tweets)

        result_count = json_response['meta']['result_count']
        count += result_count
        print(count)
        time.sleep(3.1)

    except KeyError:
        pass
    
    try:
        includes_range = range(len(json_response['includes']['tweets']))
        includes_id = []
        includes_retweet = []
            
        for i in includes_range:
            id_info = json_response['includes']['tweets'][i]['id']
            text = json_response['includes']['tweets'][i]['text']

            includes_id.append(id_info)
            includes_retweet.append(text)
            retweet_df = ''
            retweet_df = pd.DataFrame(list(zip(includes_id, includes_retweet)), 
                       columns =['retweet_id', 'text'])
    
    except KeyError:
        pass
        
    retweet_df_disk = retweet_df_disk.append(retweet_df)
    retweet_df_disk = retweet_df_disk.drop_duplicates()
                
    if 'next_token' in json_response['meta']:
        next_token = json_response['meta']['next_token']
    else:
        break

engagement = df['public_metrics'].apply(pd.Series)
final_data = pd.concat([df[['id', 'author_id','created_at', 'text']], engagement], axis=1)
final_data = final_data.reset_index(drop=True)

86


## Importing User Name from id

In [9]:
client = Twarc2(bearer_token=TOKEN)
user_id_list = final_data["author_id"]
lookup = client.user_lookup(users=user_id_list)

username = []
follower_n = []
following_n = []

for page in lookup:
    result = expansions.flatten(page)
    for user in result:
        username.append(user['username'])
        follower_n.append(user['public_metrics']['followers_count'])
        following_n.append(user['public_metrics']['following_count'])
        
final_data["username"] = username
final_data["follower_count"] = follower_n
final_data["following_count"] = following_n
final_data

id            author_id                created_at  \
0   1477769524011315210   826104766161879044  2022-01-02T22:31:24.000Z   
1   1477750903025319937  1433581386112380932  2022-01-02T21:17:25.000Z   
2   1477743788793778179           1273894620  2022-01-02T20:49:09.000Z   
3   1477740238323789831             19198704  2022-01-02T20:35:02.000Z   
4   1477737600479866884   948674925950328833  2022-01-02T20:24:33.000Z   
..                  ...                  ...                       ...   
81  1477119968584126464             18586040  2022-01-01T03:30:18.000Z   
82  1477100493830864903            436045478  2022-01-01T02:12:55.000Z   
83  1477098881880055814  1049005198860009473  2022-01-01T02:06:31.000Z   
84  1477095571777155073              9745402  2022-01-01T01:53:22.000Z   
85  1477087462564540422            539649365  2022-01-01T01:21:08.000Z   

                                                 text  retweet_count  \
0   @Iovehearts is it a pension scheme with your e...              0   
1   @AccidentAlert_K @sikikasafety @road_driving @...              1   
2   RT @PRSA_Roadsafety: Let be honest PSV drivers...              4   
3   You’ve heard of the “Great Resignation,” but a...              0   
4   @AccidentAlert_K @sikikasafety @road_driving @...              1   
..                                                ...            ...   
81  [USA] Public Relations Sr. Advisor for Infrast...              0   
82  RT @DWJr16: It has been my honor to serve as 2...              1   
83  It has been my honor to serve as 2021 @PRSAHea...              1   
84  RT @PRSA: As we close out 2021, we want to tak...              5   
85  🚨Remember to buy your tickets for our Raffle! ...              0   

    reply_count  like_count  quote_count         username  follower_count  \
0             1           0            0     _postpunkdad             675   
1             0           5            0    Missy41091460               3   
2             0           0            0          kiela_w             929   
3             0           2            0   PRSASacramento            2406   
4             0           5            0   EliasZakayo300             401   
..          ...         ...          ...              ...             ...   
81            0           0            0          pr_jobs            5837   
82            0           0            0       PRSAHealth            2293   
83            0           4            0           DWJr16             147   
84            0           0            0       alaskaegan             501   
85            0           1            0  UMDearbornPRSSA             664   

    following_count  
0               414  
1                29  
2               783  
3              1522  
4              1638  
..              ...  
81             3295  
82             1582  
83             1139  
84              437  
85              687  

[86 rows x 11 columns]

## Export the data as a excel file

In [10]:
final_data.to_excel("Twitter_data.xlsx")